# Parse

In [ ]:
import xml.etree.ElementTree as ET
ns = {"cmmn": "http://www.omg.org/spec/CMMN/20151109/MODEL"}
tree = ET.parse("/Users/wvw/Dropbox/research/projects/Montfort PM/cmmn/cap.cmmn")
ET.register_namespace("cmmn", "http://www.omg.org/spec/CMMN/20151109/MODEL")
root = tree.getroot()

In [124]:
import pprint

itemObjs = {}
planItems = root.findall(".//cmmn:planItem", namespaces=ns)

for planItem in planItems:
    defRef = planItem.attrib['definitionRef']
    
    itemObj = {
        'sentries': []
    }
    itemObjs[defRef] = itemObj
    
    reqRules = planItem.findall(".//cmmn:itemControl/cmmn:requiredRule", namespaces=ns)
    mandatory = len(reqRules) > 0
    itemObj['mandatory'] = mandatory    
    
    label = None; type = None
    if defRef.startswith("Task"):
        labelNode = root.findall(f".//cmmn:task[@id='{defRef}']", namespaces=ns)[0]
        label = labelNode.attrib['name']
        type = 'Task'
    elif defRef.startswith("Stage"):
        stage = root.findall(f".//cmmn:stage[@id='{defRef}']", namespaces=ns)[0]
        label = stage.attrib['name']
        itemObj['children'] = []
        for childPlanItem in stage.findall(".//cmmn:planItem", namespaces=ns):
            itemObj['children'].append(childPlanItem.attrib['definitionRef'])
        type = 'Stage'
    elif defRef.startswith("Milestone"):
        milestone = root.findall(f".//cmmn:milestone[@id='{defRef}']", namespaces=ns)[0]
        label = milestone.attrib['name']
        type = 'Milestone'
    else:
        continue
        
    itemObj['label'] = label
    itemObj['type'] = type
        
    # print(">", label, ("(mandatory)" if mandatory else ""))
    
    for entryCrit in planItem.findall("cmmn:entryCriterion", namespaces=ns): 
        # print("- sentry")              
        sentry = root.findall(f".//cmmn:sentry[@id='{entryCrit.attrib['sentryRef']}']", namespaces=ns)[0]
        
        sentryObj = {
            'id': sentry.attrib['id'],
            'items': [],
            'conditions': []
        }
        itemObj['sentries'].append(sentryObj)
        
        itemParts = sentry.findall(".//cmmn:planItemOnPart", namespaces=ns)
        for itemPart in itemParts:
            sentryItemObj = { 'id': itemPart.attrib['id'] }
            sourcePlanItem = root.findall(f".//cmmn:planItem[@id='{itemPart.attrib['sourceRef']}']", namespaces=ns)[0]
            sourceDefRef = sourcePlanItem.attrib['definitionRef']
            sentryItemObj['source'] = sourceDefRef
            # print("source:", sourceDefRef)
            
            events = itemPart.findall(".//cmmn:standardEvent", namespaces=ns)
            if len(events) > 0:
                eventLabel = events[0].text
                sentryItemObj['event'] = eventLabel
                # print("event:", eventLabel)
        
            sentryObj['items'].append(sentryItemObj)
        
        associations = root.findall(f".//cmmn:association[@sourceRef='{entryCrit.attrib['id']}']", namespaces=ns)
        for association in associations:
            condItemObj = { 'id': association.attrib['targetRef'] }
            textAnnotations = root.findall(f".//cmmn:textAnnotation[@id='{association.attrib['targetRef']}']", namespaces=ns)
            if len(textAnnotations) > 0:
                textAnnotation = textAnnotations[0].findall(".//cmmn:text", namespaces=ns)[0].text
                # print("condition:", textAnnotation)
                condItemObj['text'] = textAnnotation
                sentryObj['conditions'].append(condItemObj)
                
    # print("")
    
pprint.pprint(itemObjs)

{'Milestone_0gl7st9': {'label': 'hospital admission',
                       'mandatory': False,
                       'sentries': [],
                       'type': 'Milestone'},
 'Stage_1sveclw': {'children': ['Task_0kvjrmn',
                                'Task_0hsxzhz',
                                'Task_0rwjaka',
                                'Task_0akqm8t',
                                'Task_022k641'],
                   'label': '',
                   'mandatory': False,
                   'sentries': [{'conditions': [],
                                 'id': 'Sentry_1a74myc',
                                 'items': [{'event': 'occur',
                                            'id': 'PlanItemOnPart_1y7o923',
                                            'source': 'Milestone_0gl7st9'}]}],
                   'type': 'Stage'},
 'Task_022k641': {'label': 'oxygen saturation',
                  'mandatory': True,
                  'sentries': [],
                  'type': 

# Convert

In [125]:
from rdflib import Namespace, Literal, Graph, BNode, RDF, RDFS, XSD, URIRef
CM = Namespace("http://rdf.org/cmmn#")
modelNs = Namespace("http://ontario.org/qbp/cap#")

g = Graph()

In [ ]:
from convert_base import str_to_uri

for defRef, itemObj in itemObjs.items():
    planItemUri = str_to_uri(defRef, modelNs)

    if itemObj['label'].strip() != "":
        g.add((planItemUri, RDFS['label'], Literal(itemObj['label'])))
    
    g.add((planItemUri, CM['isMandatory'], Literal(itemObj['mandatory'], datatype=XSD['bool'])))

    g.add((planItemUri, RDF['type'], CM[itemObj['type']]))    
    if itemObj['type'] == 'Stage':
        for child in itemObj['children']:
            g.add((planItemUri, CM['hasChild'], str_to_uri(child, modelNs)))
            
    for sentry in itemObj['sentries']:
        sentryUri = str_to_uri(sentry['id'], modelNs)
        g.add((planItemUri, CM['hasSentry'], sentryUri))
        g.add((sentryUri, RDF['type'], CM['Sentry']))
        
        for item in sentry['items']:
            planItemPartUri = str_to_uri(item['id'], modelNs)
            g.add((sentryUri, CM['hasPlanItemPart'], planItemPartUri))
            
            sourceUri = str_to_uri(item['source'], modelNs)
            g.add((planItemPartUri, CM['hasSource'], sourceUri))
            
            eventLabel = item['event']
            g.add((planItemPartUri, CM['hasEvent'], Literal(eventLabel)))
            
        for condition in sentry['conditions']:
            conditionUri = str_to_uri(condition['id'], modelNs)
            g.add((sentryUri, CM['hasCondition'], conditionUri))
            
            g.add((conditionUri, RDFS['comment'], Literal(condition['text'])))
            
print(g.serialize(format="n3", destination="cap.ttl"))